# Saurabh Gupta


# Part 1 Split data

## Read Data

In [2]:
import pandas as pd
from google.colab import drive
import os
import functools
import math

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/ST590/HW06/scoresFull.csv'

data = pd.read_csv(file_path)
data.shape

Mounted at /content/drive


(3471, 82)

## Split Data


In [3]:
data.shape

(3471, 82)

In [4]:
# Group the data by season
season_groups = data.groupby('season')

# Iterate over each group and save to separate CSV file
for season, season_data in season_groups:
    filename = f"score_{season}.csv"
    filepath = '/content/drive/My Drive/ST590/HW06/'+filename
    season_data.to_csv(filepath, index=False)
    print(f"Season file created {filepath}")

Season file created /content/drive/My Drive/ST590/HW06/score_2002.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2003.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2004.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2005.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2006.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2007.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2008.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2009.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2010.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2011.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2012.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2013.csv
Season file created /content/drive/My Drive/ST590/HW06/score_2014.csv


# Part 2: MapReduce Idea

## MapReduce part

### group on and a numeric target variable

In [48]:
def mapping_season(season_df, group_var, target_var ):
    #sub_season_dataset = season_df.groupby(group_var)
    #return data.groupby(group_var)[target_var].agg(['sum', 'count', lambda x: (x ** 2).sum()])

    #print(season_df[group_var].unique)
    results = {}
    target_dataset = season_df.groupby(group_var)[target_var]
    #print (target_dataset.head())
    results[group_var] = find_statistics(target_dataset)
    #print (results)
    return results



### sum of the target variable across the grouping variable

In [6]:
def find_statistics(dataset) :
  results =  dataset.agg(['sum', lambda x: (x ** 2).sum(), 'count'])
  results.columns = ['sum', 'sum_squared', 'count']
  #print (results.head(10))
  return results



### write a reduce function to combine the results across the season data sets

In [7]:
# This is copied form the lecture notes as it is and it worked perfectly : )
def season_reduce(dict1, dict2):
    combined = {}
    for key in dict1.keys():
        if key in dict2:
            combined[key] = dict1[key] + dict2[key]
        else:
            combined[key] = dict1[key]
    for key in dict2.keys():
        if key not in dict1.keys():
            combined[key] = dict2[key]
    return combined

## Summarizing bit

### Find Mean and Standard Deviation of each group

In [38]:
def final_stats(combined_results):
    for key, val in list(combined_results.items())[:10]:
        mean = val['sum'] / val['count']
        temp_std_dev = ((1/(val['count'] - 1)) *  (val['sum_squared'] - val['count'] * mean ** 2))
        std_dev = temp_std_dev.apply(lambda x: math.sqrt(x))
    return [mean, std_dev]

### Load files


In [9]:
# Loading all the files dynamically from google drive

def load_season_files() :
    print ('saurabh')
    google_drive_folder = '/content/drive/My Drive/ST590/HW06/'
    print(os.listdir(google_drive_folder))

    # List of files matching the pattern
    matching_files = [file for file in os.listdir(google_drive_folder) if file.startswith('score_') and file.endswith('.csv')]
    season_data_sets = []
    for file in matching_files :
        filepath = '/content/drive/My Drive/ST590/HW06/'+file
        season_df = pd.read_csv(filepath)
        season_data_sets.append(season_df)
    return season_data_sets


### final calculation part

In [51]:
def final_function(group_var, target_var) :
    season_data_sets = load_season_files()
    # the smallest length arguemnt will be considered in mapping function.
    mapping_results = list(map(mapping_season,season_data_sets,[group_var]*len(season_data_sets),[target_var]*len(season_data_sets)))
    #print((mapping_results))
    combined_results = functools.reduce(season_reduce, mapping_results)
    #print((mapping_results))
    for key, val in list(combined_results.items())[:10]:
        print(key, ":", val)

    final_results = final_stats(combined_results)
    print(final_results[0])
    print(final_results[1])

In [52]:
# Group by awayTeam and target variable as AFinal
stats = final_function('awayTeam', 'AFinal')
# The final stats can be calulated on any group by and target variables.

saurabh
['scoresFull.csv', 'score_2002.csv', 'score_2006.csv', 'score_2007.csv', 'score_2003.csv', 'score_2010.csv', 'score_2012.csv', 'score_2008.csv', 'score_2005.csv', 'score_2004.csv', 'score_2011.csv', 'score_2009.csv', 'score_2013.csv', 'score_2014.csv']
{'awayTeam':                       sum  sum_squared  count
awayTeam                                     
Arizona Cardinals     140         3174      8
Atlanta Falcons       252         7672     10
Baltimore Ravens      158         3682      8
Buffalo Bills         168         4758      8
Carolina Panthers      95         1371      8
Chicago Bears         127         2307      8
Cincinnati Bengals    157         4155      8
Cleveland Browns      190         4746      9
Dallas Cowboys         73          777      8
Denver Broncos        195         5165      8
Detroit Lions         135         2465      8
Green Bay Packers     184         4888      8
Houston Texans        105         1813      8
Indianapolis Colts    185         46

# Part 3: Using pyspark (SQL)

This is completed in Jupyter notebook and uploaded separately

# Part 4: Using pyspark (pandas-on-spark)

This is completed in Jupyter notebook and uploaded separately